### --- Day 17: Clumsy Crucible ---

The lava starts flowing rapidly once the Lava Production Facility is operational. As you leave, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.

As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!

You land near the gradually-filling pool of lava at the base of your new lavafall. Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large crucibles on wheels.

The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.

To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible from the lava pool to the machine parts factory. To do this, you need to minimize heat loss while choosing a route that doesn't require the crucible to go in a straight line for too long.

Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.

For example:
```
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
```
Each city block is marked by a single digit that represents the amount of heat loss if the crucible enters that block. The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)

Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move at most three blocks in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.

One way to minimize heat loss is this path:
```
2>>34^>>>1323
32v>>>35v5623
32552456v>>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>
```
This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only 102.

Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, what is the least heat loss it can incur?


In [1]:
%%time
import queue

f = open("input.txt", "r")

grid = []

total = 0
for line in f:
    line = line.replace('\n', '')
    grid.append(line)
    
# Dijkstra's
visited = set()
distances = {}
prev = {}

directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

for i in range(len(grid)):
    for j in range(len(grid[0])):
        for k in range(4):
            if (j == 0 and k == 0) or (j == len(grid[0]) - 1 and k == 2) \
            or (i == 0 and k == 1) or (i == len(grid) - 1 and k == 3):
                continue
            
            for l in range(1, 4):
                distances[(i,j,k,l)] = 100000

to_visit = queue.PriorityQueue()
                
initial = (0, 0, 0, 0)
distances[initial] = 0
to_visit.put((0, initial))

while not to_visit.empty():
    _, current = to_visit.get()
    #current = min(to_visit, key=lambda n: distances[n] + 282 - n[0] - n[1])
    #to_visit.remove(current)
    visited.add(current)
    
    if current[0] == len(grid) - 1 and current[1] == len(grid[0]) - 1:
        print(current, distances[current])
        break
    
    # Get neighbors
    for d in range(4):
        next_node = None
        dire = directions[d]
        if dire == directions[current[2]]:
            if current[3] == 3:
                continue
            next_node = (current[0] + dire[0], current[1] + dire[1], d, current[3] + 1)
        elif dire[0] == -directions[current[2]][0] and dire[1] == -directions[current[2]][1]:
            continue
        else:
            next_node = (current[0] + dire[0], current[1] + dire[1], d, 1)
            
        if next_node[0] < 0 or next_node[0] >= len(grid) or next_node[1] < 0 or next_node[1] >= len(grid[0]) or next_node == None:
            continue

        # Evaluate tentative distance and swap if less
        if next_node not in visited:
            dist = distances[next_node]
            new_dist = distances[current] + int(grid[next_node[0]][next_node[1]])
            if new_dist < dist:
                distances[next_node] = new_dist
                prev[next_node] = current
                to_visit.put((new_dist + 2*len(grid) - next_node[0] - next_node[1], next_node))

(140, 140, 1, 1) 1110
CPU times: total: 141 ms
Wall time: 1.63 s


### --- Part Two ---

The crucibles of lava simply aren't large enough to provide an adequate supply of lava to the machine parts factory. Instead, the Elves are going to upgrade to ultra crucibles.

Ultra crucibles are even more difficult to steer than normal crucibles. Not only do they have trouble going in a straight line, but they also have trouble turning!

Once an ultra crucible starts moving in a direction, it needs to move a minimum of four blocks in that direction before it can turn (or even before it can stop at the end). However, it will eventually start to get wobbly: an ultra crucible can move a maximum of ten consecutive blocks without turning.

In the above example, an ultra crucible could follow this path to minimize heat loss:
```
2>>>>>>>>1323
32154535v5623
32552456v4254
34465858v5452
45466578v>>>>
143859879845v
445787698776v
363787797965v
465496798688v
456467998645v
122468686556v
254654888773v
432267465553v
```
In the above example, an ultra crucible would incur the minimum possible heat loss of 94.

Here's another example:
```
111111111111
999999999991
999999999991
999999999991
999999999991
```
Sadly, an ultra crucible would need to take an unfortunate path like this one:
```
1>>>>>>>1111
9999999v9991
9999999v9991
9999999v9991
9999999v>>>>
```
This route causes the ultra crucible to incur the minimum possible heat loss of 71.

Directing the ultra crucible from the lava pool to the machine parts factory, what is the least heat loss it can incur?


In [8]:
%%time
# Ultra Crucible
import queue

file = "input.txt"
goal = (140, 140)

test = False
if test:
    file = "test.txt"
    goal = (12, 12)

mov_min = 4
mov_max = 10
    
size = goal[0] + 1
grid = {(i,j): int(c) for i,r in enumerate(open(file)) for j,c in enumerate(r.strip())}
    
# Dijkstra's
visited = set()
distances = {}
prev = {}

directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

for key in grid:
        for k in range(4):          
            for l in range(mov_min, mov_max + 1):
                distances[key[0], key[1], k, l] = 100000

to_visit = queue.PriorityQueue()
                
initial = (0, 0, 0, 0)
distances[initial] = 0
to_visit.put((0, initial))

while not to_visit.empty():
    _, current = to_visit.get()
    visited.add(current)
    
    if (current[0], current[1]) == goal:
        print(current, distances[current])
        break
    
    # Get neighbors
    for d in range(4):
        if d == (current[2] + 2) % 4: # Backwards
            continue
            
        next_node = None
        dist = 0
        dire = directions[d]
        
        if dire == directions[current[2]]: # Forward
            if current[3] == 0: # For the initial node
                next_node = (current[0] + mov_min*dire[0], current[1] + mov_min*dire[1], d, mov_min)
                if (next_node[0], next_node[1]) not in grid:
                    continue    
                for i in range(mov_min):
                    dist += grid[current[0] + (i+1)*dire[0], current[1] + (i+1)*dire[1]]
            elif current[3] == mov_max:
                continue
            else:
                next_node = (current[0] + dire[0], current[1] + dire[1], d, current[3] + 1)
                if (next_node[0], next_node[1]) not in grid:
                    continue
                dist = grid[next_node[0], next_node[1]]
        else: # Turn L or R
            next_node = (current[0] + mov_min*dire[0], current[1] + mov_min*dire[1], d, mov_min)
            if (next_node[0], next_node[1]) not in grid:
                    continue    
            for i in range(mov_min):
                dist += grid[current[0] + (i+1)*dire[0], current[1] + (i+1)*dire[1]]

        # Evaluate tentative distance and swap if less
        if next_node not in visited:
            distance = distances[next_node]
            new_dist = distances[current] + dist
            if new_dist < distance:
                distances[next_node] = new_dist
                prev[next_node] = current
                to_visit.put((new_dist + 2*size - next_node[0] - next_node[1], next_node))

(140, 140, 1, 9) 1294
CPU times: total: 719 ms
Wall time: 4.65 s
